In [1]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import sys

import utils.cis_infer_util as cis_infer_util


/usr/local/lib/python3.6/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
jem_rna = ad.read_h5ad('data/JEM_GEX.smoothed.5NN.h5ad')
jem_atac = ad.read_h5ad('data/JEM_ATAC.smoothed.5NN.h5ad')


### Monocytes

In [3]:
jem_rna_ct = jem_rna[jem_rna.obs['cell_type'].isin(['CD14 Mono', 'CD16 Mono'])].copy()
jem_atac_ct = jem_atac[jem_atac.obs['cell_type'].isin(['CD14 Mono', 'CD16 Mono'])].copy()


In [4]:
genes = pd.read_csv('data/JEM_Mono_DEGs.csv')
genes = np.unique(genes['gene'].to_numpy())

distance = 150000
ref_tss_fn = 'utils/hg38_ref_TSS.txt'

jem_atac_ct.var_names = jem_atac_ct.var_names.str.replace(':','-',regex=False)

In [5]:
jem_rna_ct_0 = jem_rna_ct[jem_rna_ct.obs['current_severity_bin'].isin(['0'])]
jem_rna_ct_4 = jem_rna_ct[jem_rna_ct.obs['current_severity_bin'].isin(['4-5'])]
jem_rna_ct_6 = jem_rna_ct[jem_rna_ct.obs['current_severity_bin'].isin(['6-7'])]


In [6]:
jem_atac_ct_0 = jem_atac_ct[jem_atac_ct.obs['current_severity_bin'].isin(['0'])]
jem_atac_ct_4 = jem_atac_ct[jem_atac_ct.obs['current_severity_bin'].isin(['4-5'])]
jem_atac_ct_6 = jem_atac_ct[jem_atac_ct.obs['current_severity_bin'].isin(['6-7'])]


In [7]:
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']

cis_score_tmp, rna_pseudo, atac_pseudo, _ = cis_infer_util.cis_element_score(jem_rna_ct_0, jem_atac_ct_0, genes, 
                                         distance, ref_tss_fn, use_rep='model',
                                         n_pseudocells=50, return_pseudo_bulk=True)
cis_score_tmp['Method'] = 'SiaNN'
cis_score_df = pd.concat([cis_score_df,cis_score_tmp])
cis_score_df.sort_values(by=['Spearman.cor'], ascending=False).to_csv('cor_by_severity/Mono_0_spr_cor.5NN.csv')



Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Representation mode engaged, clustering based on combined SiaNN embedding...


/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


In [8]:
multiome_data = pd.concat([pd.DataFrame(rna_pseudo.X), 
                              pd.DataFrame(atac_pseudo.X)], axis=1)
multiome_data.columns = np.concatenate((rna_pseudo.var_names.to_numpy(),
                                        atac_pseudo.var_names.to_numpy()))
#NK_multiome_data['current_severity_bin'] = pseudo_label_index['cell_type'].to_numpy()
multiome_data.to_csv('cor_by_severity/Mono_0_multiome_data.5NN.csv')

In [9]:
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']

cis_score_tmp, rna_pseudo, atac_pseudo, _ = cis_infer_util.cis_element_score(jem_rna_ct_4, jem_atac_ct_4, genes, 
                                         distance, ref_tss_fn, use_rep='model',
                                         n_pseudocells=50, return_pseudo_bulk=True)
cis_score_tmp['Method'] = 'SiaNN'
cis_score_df = pd.concat([cis_score_df,cis_score_tmp])
cis_score_df.sort_values(by=['Spearman.cor'], ascending=False).to_csv('cor_by_severity/Mono_4-5_spr_cor.5NN.csv')


Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Representation mode engaged, clustering based on combined SiaNN embedding...


/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


In [10]:
multiome_data = pd.concat([pd.DataFrame(rna_pseudo.X), 
                              pd.DataFrame(atac_pseudo.X)], axis=1)
multiome_data.columns = np.concatenate((rna_pseudo.var_names.to_numpy(),
                                        atac_pseudo.var_names.to_numpy()))
#NK_multiome_data['current_severity_bin'] = pseudo_label_index['cell_type'].to_numpy()
multiome_data.to_csv('cor_by_severity/Mono_4-5_multiome_data.5NN.csv')

In [11]:
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']

cis_score_tmp, rna_pseudo, atac_pseudo, _ = cis_infer_util.cis_element_score(jem_rna_ct_6, jem_atac_ct_6, genes, 
                                         distance, ref_tss_fn, use_rep='model',
                                         n_pseudocells=50, return_pseudo_bulk=True)
cis_score_tmp['Method'] = 'SiaNN'
cis_score_df = pd.concat([cis_score_df,cis_score_tmp])
cis_score_df.sort_values(by=['Spearman.cor'], ascending=False).to_csv('cor_by_severity/Mono_6-7_spr_cor.5NN.csv')


Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Representation mode engaged, clustering based on combined SiaNN embedding...


/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


In [12]:
multiome_data = pd.concat([pd.DataFrame(rna_pseudo.X), 
                              pd.DataFrame(atac_pseudo.X)], axis=1)
multiome_data.columns = np.concatenate((rna_pseudo.var_names.to_numpy(),
                                        atac_pseudo.var_names.to_numpy()))
#NK_multiome_data['current_severity_bin'] = pseudo_label_index['cell_type'].to_numpy()
multiome_data.to_csv('cor_by_severity/Mono_6-7_multiome_data.5NN.csv')